# Part 3: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients **before** calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# Section 3.1 - Pointers to Pointers

As you know, PointerTensor objects feel just like normal tensors. In fact, they are _so much like tensors_ that we can even have pointers **to** the pointers. Check it out!

In [2]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [3]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [4]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:60883878292 -> bob:14614292715]

In [5]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:74023682811 -> alice:60883878292]

### What happened?

So, in the previous example, we created a tensor called `x` and send it to Bob, creating a pointer on our local machine (`x_ptr`). 

Then, we called `x_ptr.send(alice)` which **sent the pointer** to Alice. 

Note, this did NOT move the data! Instead, it moved the pointer to the data!! 

In [6]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{14614292715: tensor([1, 2, 3, 4])}

In [7]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{60883878292: (Wrapper)>[PointerTensor | alice:60883878292 -> bob:14614292715]}

In [8]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

(Wrapper)>[PointerTensor | me:60883878292 -> bob:14614292715]

In [9]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

tensor([1, 2, 3, 4])

### Arithmetic on Pointer -> Pointer -> Data Object

And just like with normal pointers, we can perform arbitrary PyTorch operations across these tensors

In [10]:
bob._objects

{}

In [11]:
alice._objects

{}

In [12]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [13]:
bob._objects

{22411546729: tensor([1, 2, 3, 4, 5]),
 52758901857: tensor([ 2,  4,  6,  8, 10])}

In [14]:
alice._objects

{27594038563: (Wrapper)>[PointerTensor | alice:27594038563 -> bob:22411546729],
 52758901857: (Wrapper)>[PointerTensor | alice:52758901857 -> bob:52758901857]}

In [15]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [16]:
bob._objects

{22411546729: tensor([1, 2, 3, 4, 5])}

In [17]:
alice._objects

{27594038563: (Wrapper)>[PointerTensor | alice:27594038563 -> bob:22411546729]}

In [18]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [19]:
bob._objects

{}

In [20]:
alice._objects

{}

# Section 3.2 - Pointer Chain Operations

So in the last section whenever we called a .send() or a .get() operation, it called that operation directly on the tensor on our local machine. However, if you have a chain of pointers, sometimes you want to call operations like .get() or .send() on the **last** pointer in the chain (such as sending data directly from one worker to another). To accomplish this, you want to use functions which are especially designed for this privacy preserving operation.

These operations are:

- `my_pointer2pointer.move(another_worker)`

In [21]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [22]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {23825301041: tensor([1, 2, 3, 4, 5])}
alice: {}


In [23]:
x = x.move(alice)

In [24]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

  bob: {}
alice: {45541307689: tensor([1, 2, 3, 4, 5])}


In [25]:
x

(Wrapper)>[PointerTensor | me:45541307689 -> alice:45541307689]

Excellent! Now we're equiped with the tools to perform remote **gradient averaging** using a trusted aggregator! 

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)